# Create Elastic search local

### Notes

Links:
-  url

### import

In [1]:
# imports
import os
import json
import re

import pandas as pd
import numpy as np

from elasticsearch import Elasticsearch
from tqdm import tqdm_notebook

### functions

In [16]:
def count_ipynb(folder):
    path = os.getcwd()+'\\'+folder
    try:
        dir_list = os.listdir(path)
    except:
        return "this repository doesn't exist"

    file_id = 0
    for root, dirs, files in os.walk(path):
        for file in files:
            if file.endswith(".ipynb"):
                file_id += 1
    return file_id


def code_output(cell,temp_dict):
    """
    Explanation function
    """
    if cell['outputs']==[]:
        temp_dict['output_type'] = []
        temp_dict['output'] = []
    else:
        if type(cell['outputs']) is list and len(cell['outputs'])>2:
#             print(len(cell['outputs']),temp_dict['location'],temp_dict['file_cell'],cell['outputs'])
            pass
        output_type = cell['outputs'][0]['output_type']
        temp_dict['output_type'] = output_type
        if output_type == 'stream':
            temp_dict['output'] = cell['outputs'][0]['text']
        elif output_type == 'error':
            temp_dict['output'] = cell['outputs'][0]['traceback']
        elif temp_dict['output_type'] == 'display_data':
            temp_dict['output'] = 'displayed data'
        elif temp_dict['output_type'] == 'pyout':
#             temp_dict['output'] = cell['outputs'][0]['html']
            temp_dict['output'] = 'unkown'
        elif 'data' in cell['outputs'][0].keys():
            temp_dict['output'] = list(cell['outputs'][0]['data'].values())
        elif 'text' in cell['outputs'][0].keys():
            temp_dict['output'] = cell['outputs'][0]['text']
        elif 'ename' in cell['outputs'][0].keys():
            temp_dict['output'] = cell['outputs'][0]['ename']+cell['outputs'][0]['evalue']
        else:
            temp_dict['output'] = 'unknown'
#             print(temp_dict['location'],temp_dict['file_cell'],cell)

    return temp_dict


def read_ipynb_cell(cell_id,cell_dict,file,folder,location,repo,user):
    """
    Explanation function
    """
    with open(location,encoding="utf8") as notebook:
        try: 
            data = json.load(notebook) #does the file have a correct json format
        except Exception as e:
            print(e,file)
            return cell_id,cell_dict
        
        file_cell = 0
        nbformat = data['nbformat']
        if nbformat == 4: # current nbformat
            data_cells =  data['cells']
        elif nbformat == 3: # old nbformat
            data_cells =  data['worksheets'][0]['cells']
        elif nbformat == 2: # even older format
            data_cells = data['worksheets'][0]['cells']

        for cell in data_cells:
            temp_dict = {}
            if cell['cell_type'] == 'code' and (nbformat == 3 or nbformat == 2): #cell['source'] doesn't exist within this condition, use cell['input']
                text = cell['input']
            else:
                text = cell['source']
            clean_cell = list(map(lambda s: s.strip(), text)) #remove the '\n' at the end of each string in the list         
            single_string = ' '.join(clean_cell)
            lines = len(clean_cell)

            temp_dict['file_cell'] = file_cell
            temp_dict['file'] = file
            temp_dict['nbformat'] = data['nbformat']
            temp_dict['folder'] = folder
            temp_dict['user'] = user
            temp_dict['repo'] =  repo
            temp_dict['location'] = location
            temp_dict['string'] = clean_cell
#             temp_dict['char'] = single_string
            temp_dict['lines'] = lines
            temp_dict['cell_type'] = cell['cell_type']
            if cell['cell_type'] == 'code':
                temp_dict = code_output(cell,temp_dict)

            cell_dict[cell_id] = temp_dict
            cell_id += 1
            file_cell += 1
            
    return cell_id,cell_dict


def files_to_dict(file_dict):
    """
    Explanation function
    """
    cell_id = 0
    cell_dict = {}

    for file in file_dict.keys():
        file_name = file_dict[file]['file']
        user = file_dict[file]['user']
        folder = file_dict[file]['folder']
        location = file_dict[file]['location']
        repo = file_dict[file]['repo']
        #kan ik dit niet in één regel schrijven, ff controleren nog bijv a,b,c = dict.values()

        cell_id_dict = read_ipynb_cell(cell_id,cell_dict,file_name,folder,location,repo,user)
        cell_id = cell_id_dict[0]
        cell_dict = cell_id_dict[1]
    return cell_id,cell_dict


def rec_to_actions(df):
    for record in df.to_dict(orient="records"):
        yield ('{ "index" : { "_index" : "%s", "_type" : "%s" }}'% (INDEX, TYPE))
        yield (json.dumps(record, default=int))
      

def index_marks(nrows, chunk_size):
    return range(1 * chunk_size, (nrows // chunk_size + 1) * chunk_size, chunk_size)


def split(dfm, chunk_size):
    indices = index_marks(dfm.shape[0], chunk_size)
    return np.split(dfm, indices)


def decompose_folder_name(folder_name):
    r1 = re.compile("([a-zA-Z0-9_-]+)")
    decompose = r1.findall(folder_name)
    user,repo = decompose[0],decompose[1]
    return user,repo

def create_SE_from_folder(es,folder,file_id):
    """
    Explenation
    """
    cwd = os.getcwd()
    path = cwd+'\\'+folder
    print(path)
    file_dict = {}
    
    for root, dirs, files in os.walk(path):
        for file in files:
            if file.endswith(".ipynb"):
                location = os.path.join(root,file)
#                 fn = root.split('\\')[6] # root verschilt per pc, documents\\test_thesis vs documents\\uni\\jaarx\\test_thesis
                folder_name = root.split(folder)[1].split('\\')[1]
                user,repo = decompose_folder_name(folder_name)
                temp_dict = {}
                temp_dict['file'] = file
                temp_dict['folder'] = root.split('\\')[-1]
                temp_dict['location'] = os.path.join(root,file)
                temp_dict['user'] = user
                temp_dict['repo'] = repo
                
                file_dict[file_id] = temp_dict
                file_id += 1
    
    # CREATE DICT FOR ALL CELLS
    cell_dict = files_to_dict(file_dict)[1]
 

    # CREATE DATAFRAME FROM DICT
    cell_df = pd.DataFrame.from_dict(cell_dict,orient='index')
    cell_df.index = cell_df.index.set_names(['cell_id'])
    cell_df = cell_df.fillna('empty').reset_index()
    return es,file_id,cell_df
    
    # PUT DATAFRAME INTO ELASTIC SEARCH
    for chuck in tqdm_notebook(split(cell_df, 1000)): # r = es.bulk(rec_to_actions(cell_df))
        try:
            r = es.bulk(rec_to_actions(chuck))
        except Exception as e:
            print('Bulk failed at df cell_id:',chuck.cell_id.iloc[0],'-',chuck.cell_id.iloc[-1])
    return es,file_id,cell_df
#     return cell_df,es,file_id

### Create local elastic search variable

#### CHEATSHEET Elastic Search curl's

In [8]:
# !curl "http://localhost:9200/test"
# !curl -XDELETE "localhost:9200/repos"
# !curl -XPOST "http://localhost:9200/_shutdown"

!curl "http://localhost:9200/_cat/indices?v"

health status index uuid pri rep docs.count docs.deleted store.size pri.store.size


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100    83  100    83    0     0   8300      0 --:--:-- --:--:-- --:--:--  8300


### Count .ipynb files in current repository

In [5]:
count_ipynb('repos_sample'),count_ipynb('repos')

(257, 3108)

### Put repositories into elastic search

In [6]:
HOST = 'http://localhost:9200/'
es = Elasticsearch(hosts=[HOST]) 


INDEX = "repos_sample"
TYPE = "record"

In [17]:
%%time
es=1
gallery_es,file_id,gallery_df = create_SE_from_folder(es,'repos',0)
gallery_df

C:\Users\kenne\Documents\test_thesis\repos
Expecting value: line 1 column 1 (char 0) index.ipynb
'utf-8' codec can't decode byte 0x89 in position 0: invalid start byte ml3_3d_lot.ipynb
Expecting property name enclosed in double quotes: line 26 column 1 (char 418) genome lookup.ipynb
Wall time: 27.6 s


,cell_id,file_cell,file,nbformat,folder,user,repo,location,string,lines,cell_type,output_type,output
0,0,0,Montreal Python 69.ipynb,4,(abbas-taher)Montreal-Python-69,abbas-taher,Montreal-Python-69,C:\Users\kenne\Documents\test_thesis\repos\(ab...,"[## Montreal Python 69 - Tutorial, ### By Abba...",14,markdown,empty,empty
1,1,1,Montreal Python 69.ipynb,4,(abbas-taher)Montreal-Python-69,abbas-taher,Montreal-Python-69,C:\Users\kenne\Documents\test_thesis\repos\(ab...,"[## Problem Statment: ""Aggregate Interest by I...",6,markdown,empty,empty
2,2,2,Montreal Python 69.ipynb,4,(abbas-taher)Montreal-Python-69,abbas-taher,Montreal-Python-69,C:\Users\kenne\Documents\test_thesis\repos\(ab...,"[import pandas as pd, pd.set_option('display.m...",6,code,execute_result,"[[<div>\n, <style>\n, .dataframe thead tr:..."
3,3,3,Montreal Python 69.ipynb,4,(abbas-taher)Montreal-Python-69,abbas-taher,Montreal-Python-69,C:\Users\kenne\Documents\test_thesis\repos\(ab...,[### Expected Output after Aggregating by ID:],1,markdown,empty,empty
4,4,4,Montreal Python 69.ipynb,4,(abbas-taher)Montreal-Python-69,abbas-taher,Montreal-Python-69,C:\Users\kenne\Documents\test_thesis\repos\(ab...,"[import pandas as pd, pd.set_option('display.m...",8,code,execute_result,"[[<div>\n, <style>\n, .dataframe thead tr:..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
116259,116259,1,Thin-layer-chromatography-analysis.ipynb,4,thin-layer-blob-demo,zmughal,zmughal-iperl-notebooks,C:\Users\kenne\Documents\test_thesis\repos\(zm...,"[use Modern::Perl;, use PDL;, use PDL::Image2D...",20,code,display_data,displayed data
116260,116260,2,Thin-layer-chromatography-analysis.ipynb,4,thin-layer-blob-demo,zmughal,zmughal-iperl-notebooks,C:\Users\kenne\Documents\test_thesis\repos\(zm...,[my $grey_img = $tlc_img->float->rgbtogr; # 0 ...,2,code,display_data,displayed data
116261,116261,3,Thin-layer-chromatography-analysis.ipynb,4,thin-layer-blob-demo,zmughal,zmughal-iperl-notebooks,C:\Users\kenne\Documents\test_thesis\repos\(zm...,[my $bin_img = $grey_img < 255/2; # simple thr...,3,code,display_data,displayed data
116262,116262,4,Thin-layer-chromatography-analysis.ipynb,4,thin-layer-blob-demo,zmughal,zmughal-iperl-notebooks,C:\Users\kenne\Documents\test_thesis\repos\(zm...,[my $cc_img = $bin_img->cc8compt; # connected ...,8,code,execute_result,"[[PDL: Float D [2627,488]\n, -----------------..."


In [8]:
# check if dataframe succesfolly on Elastic Search
!curl "http://localhost:9200/_cat/indices?v"

health status index        uuid                   pri rep docs.count docs.deleted store.size pri.store.size
yellow open   repos_sample 4MT5-nh4T-KzwZvdjVF6GA   1   1       6671            0     41.1mb         41.1mb


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   216  100   216    0     0   8000      0 --:--:-- --:--:-- --:--:--  8307


In [ ]:
#             return cell_id,cell_dict

In [ ]:
# file_dict = gallery_single_df
# cell_id = 0
# cell_dict = {}

# for file_id in file_dict.keys():
#     try:
#         file_name = file_dict[file_id]['file']
#         user = file_dict[file_id]['user']
#         folder = file_dict[file_id]['folder']
#         location = file_dict[file_id]['location']
#         repo = file_dict[file_id]['repo']

#         cell_id_dict = read_ipynb_cell(cell_id,cell_dict,file_name,folder,location,repo,user)
#         cell_id = cell_id_dict[0]
#         cell_dict = cell_id_dict[1]

#     except Exception as e:
#         print(e,file_dict[file_id])
# #             #kan ik dit niet in één regel schrijven, ff controleren nog bijv a,b,c = dict.values()
            
# #             cell_id_dict = read_ipynb_cell(cell_id,cell_dict,file_name,folder,location,repo,'a')
# #             cell_id = cell_id_dict[0]
# #             cell_dict = cell_id_dict[1]
# #             return cell_dict
# #         except Exception as e:
# #             print(e)
# # #             print("failed for file:",cell_dict[file]['file'],cell_dict[file])

# #     return cell_id,cell_dict

In [ ]:
# read_ipynb_cell(0,
#                 {},
#                 'genome lookup.ipynb',
#                 '(yoavram)ipython-notebooks',
#                 'C:\\Users\\kenne\\Documents\\test_thesis\\repos_sample\\(yoavram)ipython-notebooks\\genome lookup.ipynb',
#                 'ipython-notebooks',
#                 'yoavram')


# read_ipynb_cell(0,
#                 {},
#                 'genome lookup.ipynb',
#                 '(yoavram)ipython-notebooks',
#                 'C:\\Users\\kenne\\Documents\\test_thesis\\repos_sample\\(yoavram)ipython-notebooks\\genome lookup.ipynb',
#                 'ipython-notebooks',
#                 'yoavram')

In [ ]:
# loc_str = 'C:\\Documents\\test_thesis\\repos_sample\\math_and_suco\\folder2\\file'

# given = 'repos_sample'
# loc_str.split(given)[1].split('\\')[1]

In [27]:
# for chuck in split(gallery_df,1000):
#     print(chuck.cell_id.iloc[0],chuck.cell_id.iloc[-1])
    
    
# # gallery_df.iloc[0:3]

In [ ]:
# %%time

# gallery_single_df = create_SE_from_folder(es,'repos_sample',0)
# gallery_single_df

In [ ]:
# gallery_single_df[(gallery_single_df.file=="Let's Make a Barchart.ipynb") & (gallery_single_df.cell_type=="code")]
# # gallery_single_df.iloc[431].location

In [ ]:
# gallery_single_df.iloc[434].string

In [ ]:
# gallery_single_df.iloc[434].output

In [ ]:
# # get max cell in a ipynb file 
# file_name = 'diginorm.ipynb'
# gallery_single_df[gallery_single_df.file==file_name].file_cell.max()

In [ ]:
# for chuck in tqdm_notebook(split(gallery_single_df, 500)):
#     try:
#         r = es.bulk(rec_to_actions(chuck))
#     except:
#         print('failed, skip this df')

In [ ]:
# !curl "http://localhost:9200/_cat/indices?v"

### Put github repositories into elastic search

In [ ]:
# folder = 'repos'
# path = path = os.getcwd()+'//'+folder
# dir_list = os.listdir(path)
# file_id = 0
# gallery_repo_df = None

# for repo in tqdm_notebook(dir_list):
#     try:
#         df = create_SE_from_folder(repo,es,folder,file_id)

#         if gallery_repo_df is None:
#             gallery_repo_df = df
#         else:
#             gallery_repo_df = pd.concat([gallery_repo_df,df])
#         file_id = 0
#     except Exception as e:
#         print(e,repo)

# gallery_repo_df   
# # cell_df,es,file_id

In [ ]:
# HOST = 'http://localhost:9200/'
# es = Elasticsearch(hosts=[HOST]) 


# INDEX = "gallery_repos"
# TYPE = "record"

In [ ]:
# for chuck in tqdm_notebook(split(gallery_repo_df, 500)):
#     try:
#         r = es.bulk(rec_to_actions(chuck))
#     except:
#         print('failed, skip this df')

In [ ]:
# !curl "http://localhost:9200/_cat/indices?v"

### Put notebook collection into elastic search

In [ ]:
# HOST = 'http://localhost:9200/'
# es = Elasticsearch(hosts=[HOST],timeout=30,max_retries=10, retry_on_timeout=True) 


# INDEX = "restart"
# TYPE = "record"

In [ ]:
# sample_df = create_SE_from_folder('sample',es,'notebooks',0)

In [ ]:
# sample_df[-1:]['']

In [ ]:
# sample_df.drop(columns=['char'])

In [ ]:
# # set(sample_df.output_type)

# sample_df[sample_df.output_type=='display_data']

In [ ]:
# for chuck in tqdm_notebook(split(sample_df, 500)):
#     try:
#         r = es.bulk(rec_to_actions(chuck))
#     except:
#         print('failed, skip this df') # UITELGGEN WAT ER MIS GING ZONDER DE TRY/EXCEPT > IN HET VERSLAG (224)
    
    
# # for chuck in tqdm_notebook(split(sample_df.drop(columns=['char']), 10000)):
# #     r = es.bulk(rec_to_actions(chuck))

In [ ]:
# # !curl -XDELETE "localhost:9200/sample_notebooks"
# !curl "http://localhost:9200/_cat/indices?v"